In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

### 级联操作
- pd.concat, pd.append

pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```
objs
axis=0
keys
join='outer' / 'inner':表示的是级联的方式，outer会将所有的项进行级联（忽略匹配和不匹配），而inner只会将匹配的项级联到一起，不匹配的不级联
ignore_index=False
```

- 匹配级联

In [24]:
df1 = DataFrame(data=np.random.randint(0,100,size=(4,3)),columns=['A','B','C'])
df2 = DataFrame(data=np.random.randint(0,100,size=(4,4)),columns=['A','B','C','D'])
df1

,A,B,C
0,14,61,76
1,7,9,3
2,61,99,24
3,33,17,95


In [25]:
df2

,A,B,C,D
0,9,16,30,4
1,18,97,87,12
2,46,62,55,60
3,36,0,28,89


In [26]:
pd.concat((df1,df1),axis=1)

,A,B,C,A,B,C
0,14,61,76,14,61,76
1,7,9,3,7,9,3
2,61,99,24,61,99,24
3,33,17,95,33,17,95


- 不匹配级联
    - 不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致
    - 有2种连接方式：(参数join）
        - 外连接：补NaN（默认模式）
        - 内连接：只连接匹配的项（‘inner’）

In [4]:
pd.concat((df1,df2),axis=0)

,A,B,C,D
0,73,22,69,NaN
1,76,92,73,NaN
2,40,15,71,NaN
3,79,39,99,NaN
0,80,98,53,25.0
1,22,13,80,43.0
2,68,13,40,4.0
3,3,73,21,63.0


In [5]:
pd.concat((df1,df2),axis=0,join='inner')

,A,B,C
0,73,22,69
1,76,92,73
2,40,15,71
3,79,39,99
0,80,98,53
1,22,13,80
2,68,13,40
3,3,73,21


- 如果想要保留数据的完整性必须使用outer（外连接）

- append函数的使用

In [6]:
df1.append(df2)

,A,B,C,D
0,73,22,69,NaN
1,76,92,73,NaN
2,40,15,71,NaN
3,79,39,99,NaN
0,80,98,53,25.0
1,22,13,80,43.0
2,68,13,40,4.0
3,3,73,21,63.0


### 合并操作
- merge与concat的区别在于，merge需要依据某一共同列来进行合并

- 使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

- 注意每一列元素的顺序不要求一致

#### 一对一合并（与级联不同的是，他针对数据的合并）

In [7]:
from pandas import DataFrame

In [8]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],
                'group':['Accounting','Engineering','Engineering'],
                })
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [9]:
df2 = DataFrame({'employee':['Lisa','Bob','Jake'],
                'hire_date':[2004,2008,2012],
                })
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012


In [10]:
pd.merge(df1,df2,on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004


#### 一对多合并

In [11]:
df3 = DataFrame({
    'employee':['Lisa','Jake'],
    'group':['Accounting','Engineering'],
    'hire_date':[2004,2016]})
df3

,employee,group,hire_date
0,Lisa,Accounting,2004
1,Jake,Engineering,2016


In [12]:
df4 = DataFrame({'group':['Accounting','Engineering','Engineering'],
                       'supervisor':['Carly','Guido','Steve']
                })
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,Engineering,Steve


In [13]:
pd.merge(df3,df4,on='group')

,employee,group,hire_date,supervisor
0,Lisa,Accounting,2004,Carly
1,Jake,Engineering,2016,Guido
2,Jake,Engineering,2016,Steve


#### 多对多合并

In [14]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],
                 'group':['Accounting','Engineering','Engineering']})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [15]:
df5 = DataFrame({'group':['Engineering','Engineering','HR'],
                'supervisor':['Carly','Guido','Steve']
                })
df5

,group,supervisor
0,Engineering,Carly
1,Engineering,Guido
2,HR,Steve


In [16]:
pd.merge(df1,df5,how='right')

,employee,group,supervisor
0,Jake,Engineering,Carly
1,Lisa,Engineering,Carly
2,Jake,Engineering,Guido
3,Lisa,Engineering,Guido
4,NaN,HR,Steve


#### key的规范化


- 当两张表没有可进行连接的列时，可使用left_on和right_on手动指定merge中左右两边的哪一列列作为连接的列

In [17]:
df1 = DataFrame({'employee':['Bobs','Linda','Bill'],
                'group':['Accounting','Product','Marketing'],
               'hire_date':[1998,2017,2018]})
df1

,employee,group,hire_date
0,Bobs,Accounting,1998
1,Linda,Product,2017
2,Bill,Marketing,2018


In [18]:
df5 = DataFrame({'name':['Lisa','Bobs','Bill'],
                'hire_dates':[1998,2016,2007]})
df5

,name,hire_dates
0,Lisa,1998
1,Bobs,2016
2,Bill,2007


In [19]:
pd.merge(df1,df5,left_on='employee',right_on='name')

,employee,group,hire_date,name,hire_dates
0,Bobs,Accounting,1998,Bobs,2016
1,Bill,Marketing,2018,Bill,2007
